In [1]:
import os
import platform
import sys
from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
# Appending to sys paths


In [2]:
os.getcwd()

'C:\\Users\\prans\\Python files\\Kaggle Competitions\\Covid_19_object_detection'

In [3]:
sys.path.append(os.path.join(os.getcwd(), 'efficientdetv2', 'automl'))
sys.path.append(os.path.join(os.getcwd(), 'efficientdetv2', 'automl', 'efficientnetv2'))
sys.path.insert(0, os.path.join(os.getcwd(), 'efficientdetv2', 'automl', 'efficientdet'))

In [4]:
import covid_19_dataloader
import hparams_config
import utils
from keras import tfmot
from keras import train_lib
from keras import util_keras

In [23]:
from object_detection import argmax_matcher

In [31]:
matches = tf.constant([[0.3,0.4,0.6,0.45,0.3],[0.3,0.4,0.2,0.1,0.8],[0.2,0.4,0.35,0.6,0.1]])
matches

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[0.3 , 0.4 , 0.6 , 0.45, 0.3 ],
       [0.3 , 0.4 , 0.2 , 0.1 , 0.8 ],
       [0.2 , 0.4 , 0.35, 0.6 , 0.1 ]], dtype=float32)>

In [32]:
obj = argmax_matcher.ArgMaxMatcher(0.5, 0.5, True, True)

In [33]:
obj._match(matches)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-1, -1,  0,  2,  1])>

In [5]:
val_file_pattern = 'D:\\Datasets\\siim_covid19_detection\\1080px\\tfrecords\\fold_0\\valid\\*'
train_file_pattern = 'D:\\Datasets\\siim_covid19_detection\\1080px\\tfrecords\\fold_0\\train\\*'
val_json_file = 'D:\Datasets\siim_covid19_detection\1080px\object_detection_files\files_fold_0\valid\object_detection_info.json'

In [6]:
def setup_model(model, config):
    """Build and compile model."""
    model.build((None, *config.image_size, 3))
    model.compile(
          steps_per_execution=config.steps_per_execution,
          optimizer=train_lib.get_optimizer(config.as_dict()),
          loss={
              train_lib.BoxLoss.__name__:
                  train_lib.BoxLoss(
                      config.delta, reduction=tf.keras.losses.Reduction.NONE),
              train_lib.BoxIouLoss.__name__:
                  train_lib.BoxIouLoss(
                      config.iou_loss_type,
                      config.min_level,
                      config.max_level,
                      config.num_scales,
                      config.aspect_ratios,
                      config.anchor_scale,
                      config.image_size,
                      reduction=tf.keras.losses.Reduction.NONE),
              train_lib.FocalLoss.__name__:
                  train_lib.FocalLoss(
                      config.alpha,
                      config.gamma,
                      label_smoothing=config.label_smoothing,
                      reduction=tf.keras.losses.Reduction.NONE),
              tf.keras.losses.CategoricalCrossentropy.__name__:
                  tf.keras.losses.CategoricalCrossentropy(
                      from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
          })
    return model


In [7]:
def init_experimental(config):
    """Serialize train config to model directory."""
    tf.io.gfile.makedirs(config.model_dir)
    config_file = os.path.join(config.model_dir, 'config.yaml')
    if not tf.io.gfile.exists(config_file):
        tf.io.gfile.GFile(config_file, 'w').write(str(config))

In [8]:
model_dir = os.path.join('C:\\Users\\prans\\Python files\\Kaggle Competitions\\Covid_19_object_detection', 'object_detection', 'models')
val_file_pattern = 'D:\\Datasets\\siim_covid19_detection\\1080px\\tfrecords\\fold_0\\valid\\*'
train_file_pattern = 'D:\\Datasets\\siim_covid19_detection\\1080px\\tfrecords\\fold_0\\train\\*'
val_json_file = r'D:\Datasets\siim_covid19_detection\1080px\object_detection_files\files_fold_0\valid\object_detection_info.json'
model_name = 'efficientdetv2-s'
hparams = ''
num_epochs = 1
batch_size = 1
num_of_examples_per_epoch = 10000
mode = 'traineval'
debug = False
use_fake_data = False
eval_samples = 2000
steps_per_execution = 1

In [9]:
def main():
    config = hparams_config.get_detection_config(model_name)
    config.override(hparams)
    config.num_epochs = num_epochs
    config.image_size = utils.parse_image_size(config.image_size)
    steps_per_epoch = num_of_examples_per_epoch//batch_size
    params = dict(
        model_name = model_name,
        steps_per_execution = steps_per_execution,
        model_dir = model_dir,
        steps_per_epoch = 1,
        batch_size = batch_size,
        tf_random_seed = 42,
        debug = debug,
        val_json_file = val_json_file)
    config.override(params, True)
    
    def get_dataset(is_training, config):
        file_pattern = (
            train_file_pattern
            if is_training else val_file_pattern)
        if not file_pattern:
            raise ValueError('No matching files.')

        return covid_19_dataloader.InputReader(
            file_pattern,
            is_training=is_training,
            use_fake_data=use_fake_data,
            max_instances_per_image=config.max_instances_per_image,
            debug=debug)(config.as_dict())

    model = train_lib.EfficientDetNetTrain(config=config)
    model = setup_model(model, config)
    if debug:
        tf.config.run_functions_eagerly(True)
    
    if 'train' in mode:
        val_dataset = get_dataset(False, config) if 'eval' in mode else None
        model.fit(get_dataset(True, config),
                 epochs = config.num_epochs,
                 steps_per_epoch = steps_per_epoch,
                 callbacks = train_lib.get_callbacks(config.as_dict(), 
                                                      val_dataset),
                 validation_data = val_dataset,
                 validation_steps = (eval_samples//batch_size))
    else:
        for ckpt in tf.train.checkpoints_iterator(mode_dir, min_interval_secs = 180):
            logging.info('Starting to evaluate.')
            try:
                current_epoch = int(os.path.basename(ckpt).split('-')[1])
            except IndexError:
                current_epoch = 0
            
            val_dataset = get_dataset(False, config)
            logging.info('start loading model.')
            model.load_weights(tf.train.latest_checkpoint(model_dir))
            logging.info('finish loading model.')

In [10]:
#main()

In [11]:
records = tf.data.Dataset.list_files(
        train_file_pattern, shuffle=True, seed=1)

In [12]:
config = hparams_config.get_detection_config(model_name)
config.override(hparams)
config.num_epochs = num_epochs
config.image_size = utils.parse_image_size(config.image_size)
steps_per_epoch = num_of_examples_per_epoch//batch_size
params = dict(
    model_name = model_name,
    steps_per_execution = steps_per_execution,
    model_dir = model_dir,
    steps_per_epoch = 1,
    batch_size = batch_size,
    tf_random_seed = 42,
    debug = debug,
    val_json_file = val_json_file)
config.override(params, True)

In [13]:
def _prefetch_dataset(filename):
    if params.get('dataset_type', None) == 'sstable':
        pass
    else:
        dataset = tf.data.TFRecordDataset(filename).prefetch(1)
    return dataset

dataset = records.interleave(
        _prefetch_dataset, num_parallel_calls=tf.data.AUTOTUNE)

In [14]:
@property
def dataset_options():
    options = tf.data.Options()
    options.experimental_deterministic = debug or not True
    options.experimental_optimization.map_parallelization = True
    options.experimental_optimization.parallel_batch = True
    return options


In [15]:
# dataset = dataset.with_options(dataset_options)
if True:
    dataset = dataset.shuffle(64, seed=42)

In [16]:
sample_file_pattern = next(records.as_numpy_iterator())

In [17]:
sample_file_pattern.decode()

'D:\\Datasets\\siim_covid19_detection\\1080px\\tfrecords\\fold_0\\train\\train-16-of-32.tfrecord'

In [18]:
reader = covid_19_dataloader.InputReader(
            sample_file_pattern,
            is_training=True,
            use_fake_data=use_fake_data,
            max_instances_per_image=config.max_instances_per_image,
            debug=debug)

In [19]:
from collections import OrderedDict

In [20]:
demo_dic = OrderedDict()

In [21]:
demo_dic[3] = tf.zeros((64,64,9)).numpy()
demo_dic[4] = tf.random.normal((32,32,9)).numpy()

In [33]:
list(demo_dic.items())

[(3,
  array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         ...,
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
    

In [32]:
tf.ragged.constant(tf.nest.flatten(demo_dic))[0].to_tensor()

ValueError: all scalar values must have the same nesting depth

In [23]:
tf.RaggedTensor.dtype

In [19]:
dataset = reader(config)

<bound method InputReader.dataset_parser of <covid_19_dataloader.InputReader object at 0x000002A53B9B65E0>>
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [20]:
next(dataset.as_numpy_iterator())

(array([[[-2.117904  , -2.0357144 , -1.8044444 ],
         [-2.117904  , -2.0357144 , -1.8044444 ],
         [-2.117904  , -2.0357144 , -1.8044444 ],
         ...,
         [-2.0665298 , -1.9831933 , -1.7521569 ],
         [-2.0665298 , -1.9831933 , -1.7521569 ],
         [-2.0741556 , -1.9909893 , -1.7599183 ]],
 
        [[-2.117904  , -2.0357144 , -1.8044444 ],
         [-2.117904  , -2.0357144 , -1.8044444 ],
         [-2.117904  , -2.0357144 , -1.8044444 ],
         ...,
         [-2.0665298 , -1.9831933 , -1.7521569 ],
         [-2.0665298 , -1.9831933 , -1.7521569 ],
         [-2.0741556 , -1.9909893 , -1.7599183 ]],
 
        [[-2.117904  , -2.0357144 , -1.8044444 ],
         [-2.117904  , -2.0357144 , -1.8044444 ],
         [-2.117904  , -2.0357144 , -1.8044444 ],
         ...,
         [-2.0665298 , -1.9831933 , -1.7521569 ],
         [-2.0665298 , -1.9831933 , -1.7521569 ],
         [-2.0741556 , -1.9909893 , -1.7599183 ]],
 
        ...,
 
        [[-2.039906  , -1.9559754 

In [25]:
for i in dataset.take(1):
    print(i)

Starting to flatten
flattenedStarting to flatten

Starting to flatten
Starting to flatten
Starting to flatten
Starting to flatten
Starting to flatten
Starting to flatten
Starting to flatten
Starting to flatten
Starting to flattenStarting to flatten
Starting to flatten
Starting to flatten
flattened

flattened
Starting to flatten
flattenedflattened

flattened
Starting to flattenflattened
flattened
flattenedflattened
flattened

flattened

flattenedflattened

flattened
flattened
Starting to flatten
flattened


InvalidArgumentError: ValueError: TypeError: object of type 'RaggedTensor' has no len()

Traceback (most recent call last):

  File "C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\ops\script_ops.py", line 247, in __call__
    return func(device, token, args)

  File "C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\ops\script_ops.py", line 144, in __call__
    outputs = [

  File "C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\ops\script_ops.py", line 145, in <listcomp>
    _maybe_copy_to_context_device(self._convert(x, dtype=dtype),

  File "C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\ops\script_ops.py", line 124, in _convert
    return ops.convert_to_tensor(value, dtype=dtype)

  File "C:\Users\prans\Python files\Kaggle

In [23]:
def map_fn(value):
      return self.dataset_parser(value, example_decoder, anchor_labeler, params)
    # image, cls_targets, box_targets, num_positives, source_id, 
    #           image_scale, boxes, areas, classes
    # dataset = dataset.map(lambda value: tf.py_function(map_fn, inp = [value], Tout = [tf.float32, tf.int32, tf.float32, tf.float32, tf.string, tf.float32, tf.float32, tf.float32, tf.float32]), num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(map_fn, tf.data.AUTOTUNE)

NameError: in user code:

    <ipython-input-23-c118f4d3c386>:2 map_fn  *
        return self.dataset_parser(value, example_decoder, anchor_labeler, params)

    NameError: name 'self' is not defined


In [16]:
#serialized_file = next(tf.data.TFRecordDataset(records).as_numpy_iterator())

In [17]:
# for i in tf.data.TFRecordDataset(records).take(1):
#     print(i.numpy())

In [22]:
tf.convert_to_tensor(tf.nest.flatten({'a':tf.constant(3), 'b': tf.constant(5)}))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 5])>

In [14]:
from object_detection import tf_example_decoder

In [15]:
example_encoder = tf_example_decoder.TfExampleDecoder()

In [16]:
decoded_example = example_encoder.decode(serialized_file)
decoded_example

{'image': <tf.Tensor: shape=(1080, 1080, 3), dtype=uint8, numpy=
 array([[[ 47,  47,  47],
         [ 52,  52,  52],
         [ 56,  56,  56],
         ...,
         [ 47,  47,  47],
         [ 47,  47,  47],
         [ 40,  40,  40]],
 
        [[ 50,  50,  50],
         [ 54,  54,  54],
         [ 57,  57,  57],
         ...,
         [ 53,  53,  53],
         [ 49,  49,  49],
         [ 42,  42,  42]],
 
        [[ 51,  51,  51],
         [ 58,  58,  58],
         [ 60,  60,  60],
         ...,
         [ 54,  54,  54],
         [ 52,  52,  52],
         [ 44,  44,  44]],
 
        ...,
 
        [[178, 178, 178],
         [183, 183, 183],
         [186, 186, 186],
         ...,
         [158, 158, 158],
         [153, 153, 153],
         [150, 150, 150]],
 
        [[178, 178, 178],
         [181, 181, 181],
         [186, 186, 186],
         ...,
         [156, 156, 156],
         [152, 152, 152],
         [148, 148, 148]],
 
        [[176, 176, 176],
         [177, 177, 177],
   

In [17]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [63]:
print(config)

act_type: swish
alpha: 0.25
anchor_scale: 4.0
apply_bn_for_resampling: true
aspect_ratios:
- 1.0
- 2.0
- 0.5
autoaugment_policy: null
backbone_config:
    include_top: false
    pretrained: true
    pretrained_path: !!python/object/apply:pathlib.WindowsPath
    - C:\
    - Users
    - prans
    - Python files
    - Kaggle Competitions
    - Covid_19_object_detection
    - checkpoints
    - train
    - efficientNetV2-s
    - 512px
    with_endpoints: true
backbone_name: efficientnetv2-s
batch_size: 32
box_class_repeats: 3
box_loss_weight: 50.0
ckpt_var_scope: null
clip_gradients_norm: 10.0
conv_after_downsample: false
conv_bn_act_pattern: false
data_format: channels_last
dataset_type: null
debug: false
delta: 0.1
drop_remainder: true
first_lr_drop_epoch: 5.0
fpn_cell_repeats: 3
fpn_config: null
fpn_name: bifpn
fpn_num_filters: 64
fpn_weight_method: null
gamma: 1.5
grad_checkpoint: false
grid_mask: false
heads:
- object_detection
image_size: !!python/tuple
- 512
- 512
img_summary_steps: 

In [ ]:
dataset = reader(config)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




data: [[150.34328 600.82965 670.186   919.83746]
 [148.03798 296.0633  304.79764 508.73514]
 [470.77847 177.38478 656.35425 522.97656]], data_shape: [3 4]
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(17, shape=(), dtype=int32) 4
data: [[0.14]
 [0.03]
 [0.05]], data_shape: [3 1]
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(17, shape=(), dtype=int32) 1
data: [[1.]
 [1.]
 [1.]], data_shape: [3 1]
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(17, shape=(), dtype=int32) 1
Before parsing: b'\n\xe8\xd3\x17\n\x1d\n\x11image/object/area\x12\x08\x12\x06\n\x04\n\xd7#=\n\x1b\n\x0fimage/source_id\x12\x08\n\x06\n\x045943\n"\n\x16image/object/bbox/ymax\x12\x08\x12\x06\n\x04P\x03\x17?\n"\n\x16image/object/bbox/xmax\x12\x08\x12\x06\n\x04\xc7\xef\xad>\n\xfd\xcf\x17\n\rimage/encoded\x12\xea\xcf\x17\n\xe6\xcf\x17\n\xe2\xcf\x17\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x048\x00\x00\x048\x08\x00\x00\x00\x00\xc9{3S\x00\x01\x00\x00IDATx\x9c\xec\xfdK\x93$I\x92%\xea\x9dsXD\xcd#\xab\xfbv\x0f.\x064+\x10\xfe\x

In [ ]:
for i in dataset.take(1):
    a = i

In [ ]:
def map_fn(value):
    return reader.dataset_parser(value, example_encoder, anchor_labeler, config)

In [ ]:
tf.data.Dataset.from_tensor_slices()

In [ ]:
for value in dataset.as_numpy_iterator():
    map_fn(value)

In [ ]:
from keras import anchors

In [ ]:
input_anchors = anchors.Anchors(3, 7,
                                config['num_scales'],
                                config['aspect_ratios'],
                                config['anchor_scale'],
                                config['image_size'])
anchor_labeler = anchors.AnchorLabeler(input_anchors, config['num_classes'])

In [ ]:
image, cls_targets, box_targets, num_positives, source_id, image_scale, boxes, areas, classes, input_processor = reader.dataset_parser(serialized_file, example_encoder, anchor_labeler, config)

In [ ]:
input_processor._image_scale

In [ ]:
tf.where(cls_targets[5] >= 0)

In [ ]:
cls_targets[5][8,4,5]